In [ ]:
# !pip install ydata-profiling
# !pip install ipywidgets
# !pip install seaborn
# !pip install hvplot
# !pip install ipympl
# !pip install plotly
# !pip install jupyter_bokeh
# ! pip install shap
# !pip install dash dash-bootstrap-components
# !pip install dask
# !pip install pyarrow
# !pip install zipcodes

In [ ]:
%cd ..

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import hvplot.pandas
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import shap

hvplot.extension('plotly')
output_notebook()
# %matplotlib widget
%matplotlib ipympl
%matplotlib inline
plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')
shap.initjs()

In [ ]:
# ask for zip code in this file
data_path = "data/GoMaterials_Historical_Inventory_20250225.csv"

In [ ]:
supplier_name_to_zip_code = pd.read_csv("data/supplier_name_zip_code.csv")
supplier_name_to_zip_code = supplier_name_to_zip_code.drop_duplicates()
supplier_name_to_zip_code.head()

In [ ]:
data = pd.read_csv(
    data_path, 
    delimiter=",", 
    encoding="latin1", 
    # on_bad_lines="warn", 
    on_bad_lines='warn', 
    engine='python'
    )

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
UPDATE_DATE = "Updated_date"
SUPPLIER_ADDRESS = "Supplier_address"
SUPPLIER_REGION = "Supplier_region"
SUPPLIER_NAME = "Supplier_name"
SUPPLIER_ID = "Supplier_id"
PRICE = "Price"
PLANT_NAME = "Latin_name"
SUPPLIER_ZIP_CODE = "Supplier_Zip_Postal_Code"

SELECTED_COLUMNS = [UPDATE_DATE, SUPPLIER_ADDRESS, SUPPLIER_REGION, SUPPLIER_NAME, SUPPLIER_ID, PRICE, PLANT_NAME]
data = data[SELECTED_COLUMNS]

supplier_name_to_zip_code = supplier_name_to_zip_code.rename(columns={"Supplier Name": SUPPLIER_NAME})

# Data cleaning

In [ ]:
data = data[~data[SUPPLIER_REGION].isin(["Ontario", "Quebec"])]

In [ ]:
len(data)

In [ ]:
data[SUPPLIER_NAME].nunique()

In [ ]:
len(supplier_name_to_zip_code)

In [ ]:
data = data.join(supplier_name_to_zip_code.set_index(SUPPLIER_NAME), on=SUPPLIER_NAME)

In [ ]:
# from ollama import chat
# from ollama import ChatResponse

# def get_zip_code(address):
#     response = chat(model='llama3', messages=[
#         {
#             'role': 'user',
#             'content': f"""
#                 Return the postal code of the following address {address}? Just return the postal code. Dont' make a sentence If you ignore the response, output N/A.
#                 """,
#         },
#     ])
#     return response.message.content

# data[data[SUPPLIER_ZIP_CODE].isna()][SUPPLIER_ZIP_CODE] = data[data[SUPPLIER_ZIP_CODE].isna()][SUPPLIER_ADDRESS].apply(get_zip_code)

In [ ]:
data = data[data[SUPPLIER_ZIP_CODE].notna()]

In [ ]:
data[SUPPLIER_NAME].nunique()

In [ ]:
import zipcodes

LONGITUDE = "Longitude"
LATITUDE = "Latitude"

MISSING_ZIP_CODES_COORDINATES = {
    "8344":{
        "lat": 39.55,
        "long": -75.02
    },
    "8313":{
        "lat": 39.52,
        "long": -75.22
    },
    "8527":{
        "lat": 40.12,
        "long": -74.30
    },
    "7726":{
        "lat": 40.28,
        "long": -74.34
    },
    "8343":{
        "lat": 39.64,
        "long": -75.15
    },
    "8302":{
        "lat": 39.37,
        "long": -75.16
    },
}

def get_lat_long_from_zip_code(input_zip_code):
    try:
        matched_area = zipcodes.matching(input_zip_code)[0]
    except:
        return MISSING_ZIP_CODES_COORDINATES[input_zip_code]
    return matched_area

In [ ]:
coordinates = data[SUPPLIER_ZIP_CODE].apply(get_lat_long_from_zip_code)

data[LATITUDE] = coordinates.apply(lambda x: float(x['lat']))
data[LONGITUDE] = coordinates.apply(lambda x: float(x['long']))


In [ ]:
len(data)

In [ ]:
data["Price"] = pd.to_numeric(data["Price"], errors='coerce')
data = data[data["Price"].notna()]
len(data)

In [ ]:
data["Updated_date"] = pd.to_datetime(data["Updated_date"]).dt.strftime('%Y-%m')
data["Updated_date"].head()

In [ ]:
data["Updated_date"].min(), data["Updated_date"].max()

In [ ]:
data["Updated_date"].nunique()

In [ ]:
data["Updated_date"].hist()

In [ ]:
data.sample(5).head()

In [ ]:
data.reset_index().to_parquet("data/processed_inventory_data.parquet")